# NBA 2021 Project
### Francis Peng, Annie Seo

# Introduction and Motivation

- Contract/value/net-worth
- Shot selection, 2s, 3s, midrange, layup, etc.
- Predicting outcomes/win percentage based on midgame stats of indiivudal players.


# Data and Code Setup

We will use the following API which scrapes NBA basketball statistics from basketball-reference.com
https://github.com/vishaalagartha/basketball_reference_scraper

In [13]:
# !pip install basketball-reference-scraper
# !pip install unidecode

In [7]:
# Boilerplate
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import scipy.stats as stats
from scipy.optimize import minimize

In [12]:
# Load basketball-reference libraries
from basketball_reference_scraper.teams import get_roster, get_team_stats, get_opp_stats, get_roster_stats, get_team_misc
from basketball_reference_scraper.players import get_stats, get_game_logs, get_player_headshot
from basketball_reference_scraper.seasons import get_schedule, get_standings
from basketball_reference_scraper.box_scores import get_box_scores
from basketball_reference_scraper.pbp import get_pbp
from basketball_reference_scraper.shot_charts import get_shot_chart

# Methodology and Analysis

# Conclusion